In [5]:
# 라이브러리 불러오기
import os
from dotenv import load_dotenv
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd

Embedding

In [ ]:
# from langchain_openai import OpenAI, OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
# -> 로컬에서 작업할 때에는 위와 같은 API 호출방식이 아닌 아래와 같이 HuggingFaceEmbeddings를 사용
# sentence-transformers/all-MiniLM-L6-v2를 사용


from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"     # 한국어 지원 모델로 변경 가능 (예: BAAI/bge-m3
)


query_result = embeddings.embed_query('저는 배가 고파요')
print(query_result)
print(len(query_result))

[-0.016300534829497337, 0.08601358532905579, 0.035331521183252335, -0.03424869477748871, -0.000330631883116439, 0.00996420718729496, 0.1301051825284958, 0.02236033044755459, -0.013863354921340942, -0.041913650929927826, 0.09747382998466492, -0.05304761976003647, 0.08991933614015579, 0.0061574713326990604, 0.08977439999580383, -0.052820608019828796, 0.012548998929560184, 0.02062290720641613, -0.073173888027668, -0.023939169943332672, 0.06103256717324257, 0.04310121759772301, 0.011418970301747322, 0.03744944930076599, -0.08106309920549393, 0.033687934279441833, 0.00046312823542393744, 0.004082815255969763, 0.10455012321472168, 0.016728434711694717, -0.03324294462800026, 0.057545244693756104, 0.05144355818629265, 0.03899669274687767, -0.05049128830432892, 0.0010526623809710145, -0.07656817883253098, -0.02270135097205639, -0.019967053085565567, 0.07372038066387177, -0.17780642211437225, -0.10527726262807846, 0.04590520262718201, -0.10440733283758163, 0.09605521708726883, -0.063241176307201

Cosine Similarity 계산

In [8]:
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '부동산 시장이 점점 더 복잡해지고 있어요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 변동했어요',
]
df = pd.DataFrame(data, columns=['text'])
df

,text
0,주식 시장이 급등했어요
1,시장 물가가 올랐어요
2,전통 시장에는 다양한 물품들을 팔아요
3,부동산 시장이 점점 더 복잡해지고 있어요
4,저는 빠른 비트를 좋아해요
5,최근 비트코인 가격이 많이 변동했어요


In [9]:
def get_embedding(text):
    return embeddings.embed_query(text)

df['embedding'] = df['text'].apply(lambda x: get_embedding(x))
df

,text,embedding
0,주식 시장이 급등했어요,"[-0.05460004881024361, 0.1230602115392685, -0...."
1,시장 물가가 올랐어요,"[-0.039983246475458145, 0.1241038367152214, 0...."
2,전통 시장에는 다양한 물품들을 팔아요,"[0.015452361665666103, 0.03094390779733658, -0..."
3,부동산 시장이 점점 더 복잡해지고 있어요,"[-0.018047843128442764, 0.05511883273720741, -..."
4,저는 빠른 비트를 좋아해요,"[-0.027693873271346092, 0.12685897946357727, 0..."
5,최근 비트코인 가격이 많이 변동했어요,"[-0.021346813067793846, 0.07299348711967468, 0..."


In [10]:
def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

# 주어진 쿼리와 가장 유사한 상위 3개의 문서를 반환하는 함수
def return_answer_candidate(df, query):
    query_embedding = get_embedding(query)    # 쿼리 텍스트를 임베딩 벡터로 변환
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(np.array(x), np.array(query_embedding)))    # 문서 ↔ 쿼리 유사도 계산
    top_three_doc = df.sort_values("similarity", ascending=False).head(3)    # 정렬, 상위 3개 문서 선택
    return top_three_doc

similar_docs = return_answer_candidate(df, "부동산 가격이 올랐나요?")
similar_docs

,text,embedding,similarity
0,주식 시장이 급등했어요,"[-0.05460004881024361, 0.1230602115392685, -0....",0.849718
1,시장 물가가 올랐어요,"[-0.039983246475458145, 0.1241038367152214, 0....",0.817652
5,최근 비트코인 가격이 많이 변동했어요,"[-0.021346813067793846, 0.07299348711967468, 0...",0.753791


Document loader

In [12]:
# Webpage to Text
from langchain_community.document_loaders import WebBaseLoader

# 단일 URL 로드
loader = WebBaseLoader("https://ko.wikipedia.org/wiki/인공지능")

# 다중 URL 로드
multi_loader = WebBaseLoader([
    "https://ko.wikipedia.org/wiki/인공지능",
    "https://ko.wikipedia.org/wiki/머신러닝",
])

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
# 단일 URL 문서 불러오기
single_documents = loader.load()
print(f"단일 URL에서 불러온 문서 개수: {len(single_documents)}")
print(single_documents[0].metadata)
print(single_documents[0].page_content[:30])  # 첫 30자 출력

단일 URL에서 불러온 문서 개수: 1
{'source': 'https://ko.wikipedia.org/wiki/인공지능', 'title': '인공지능 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}




인공지능 - 위키백과, 우리 모두의 백과사전




In [18]:
# 다중 URL 문서 불러오기
multi_documents = multi_loader.load()
print(f"다중 URL에서 불러온 문서 개수: {len(multi_documents)}")
print(multi_documents[0].metadata)
print(multi_documents[1].metadata)


다중 URL에서 불러온 문서 개수: 2
{'source': 'https://ko.wikipedia.org/wiki/인공지능', 'title': '인공지능 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}
{'source': 'https://ko.wikipedia.org/wiki/머신러닝', 'title': '기계 학습 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}


In [ ]:
# Recursive URL Loader
from langchain_community.document_loaders import RecursiveUrlLoader

# 특정 패턴의 링크만 따라가고, 특정 디렉토리는 제외하는 예제
loader = RecursiveUrlLoader("https://python.langchain.com/docs/introduction/",
                            max_depth=2,        # 2단계 깊이까지 링크 추적
                            prevent_outside=True,       # 외부 링크로 나가지 않음
                            link_regex = r'.*?smith.*?',    # 'smith'가 포함된 링크만 추적
                            exclude_dirs=['https://python.langchain.com/docs/concepts/'])   # 제외할 디렉토리 목록
docs = loader.load()

In [ ]:
docs[0].metadata

In [ ]:
concept_docs = [doc for doc in docs if 'concepts' in doc.page_content]
for doc in concept_docs:
    print(doc.metadata['source'])

In [ ]:
docs[0].page_content[:1000]

PDF loader

In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader

In [23]:
%%time
loader = PyPDFLoader("./data/2024_KB_부동산_보고서_최종.pdf")
pages = loader.load_and_split()
print('청크의 수:', len(pages))

청크의 수: 83
CPU times: total: 12.4 s
Wall time: 12.7 s


In [24]:
pages[10]

Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'title': 'Morning Meeting', 'author': '손은경', 'moddate': '2024-03-04T15:30:01+09:00', 'source': './data/2024_KB_부동산_보고서_최종.pdf', 'total_pages': 84, 'page': 11, 'page_label': '12'}, page_content='5 \n2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 \n \n표Ⅰ-1. 공급 확대 정책 주요 내용 \n날짜 주요 내용 \n2023년 9월 \n공공 부문 공급물량 확대: 3기 신도시를 포함한 공급물량 확대 및 조기 공급 방안 마련 \n사업 여건 개선: 전매 제한 완화 및 규제 정상화, 조기 인허가 인센티브 및 절차 개선, 공사비 증액 \n기준 마련 및 인력 확충, 분양 사업의 임대 사업 전환 촉진 \n원활한 자금 지원: PF대출 보증 확대, 중도금 대출 지원 \n비아파트 자금 조달 지원 및 규제 개선 \n도심 공급 기반 확충: 정비사업 절차 및 소규모 사업 사업성 개선 \n2024년 1월 \n도심 공급 확대: 재건축·재개발 패스트트랙 도입 및 재건축 부담금 합리화, 1기 신도시 재정비사업의 \n신속하고 내실 있는 계획 수립, 소규모 정비·도심 복합 사업 속도 개선 및 자금 지원 강화 \n다양한 유형의 주택 공급 확대: 도시·건축 규제 완화 및 세제·금융 지원, 등록 임대 사업 여건 개선 \n및 기업형 장기임대 활성화, 신축 매입 약정 확대, 전세 사기 예방 및 피해 지원 \n신도시 등 공공주택 공급: 2024년 공공주택 14만 호 이상 공급, 신도시 조성 속도 제고 \n건설 경기 활력 회복: PF대출 지원 및 유동성 지원, 공공 임대 참여 지분 조기 매각, 민간 사업

In [25]:
pages[10].metadata

{'producer': 'Microsoft® Word 2016',
 'creator': 'Microsoft® Word 2016',
 'creationdate': '2024-03-04T15:30:01+09:00',
 'title': 'Morning Meeting',
 'author': '손은경',
 'moddate': '2024-03-04T15:30:01+09:00',
 'source': './data/2024_KB_부동산_보고서_최종.pdf',
 'total_pages': 84,
 'page': 11,
 'page_label': '12'}

In [26]:
print(pages[10].page_content)

5 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
표Ⅰ-1. 공급 확대 정책 주요 내용 
날짜 주요 내용 
2023년 9월 
공공 부문 공급물량 확대: 3기 신도시를 포함한 공급물량 확대 및 조기 공급 방안 마련 
사업 여건 개선: 전매 제한 완화 및 규제 정상화, 조기 인허가 인센티브 및 절차 개선, 공사비 증액 
기준 마련 및 인력 확충, 분양 사업의 임대 사업 전환 촉진 
원활한 자금 지원: PF대출 보증 확대, 중도금 대출 지원 
비아파트 자금 조달 지원 및 규제 개선 
도심 공급 기반 확충: 정비사업 절차 및 소규모 사업 사업성 개선 
2024년 1월 
도심 공급 확대: 재건축·재개발 패스트트랙 도입 및 재건축 부담금 합리화, 1기 신도시 재정비사업의 
신속하고 내실 있는 계획 수립, 소규모 정비·도심 복합 사업 속도 개선 및 자금 지원 강화 
다양한 유형의 주택 공급 확대: 도시·건축 규제 완화 및 세제·금융 지원, 등록 임대 사업 여건 개선 
및 기업형 장기임대 활성화, 신축 매입 약정 확대, 전세 사기 예방 및 피해 지원 
신도시 등 공공주택 공급: 2024년 공공주택 14만 호 이상 공급, 신도시 조성 속도 제고 
건설 경기 활력 회복: PF대출 지원 및 유동성 지원, 공공 임대 참여 지분 조기 매각, 민간 사업장 
공공 인수, 재정 조기 집행 및 민자 사업 확대 
 
자료: 국토교통부 보도자료 요약 
 
■ 2024년 주택시장 3대 변수는 공급과 금리, 그리고 정책 
주택시장이 상승과 하락을 반복하면서 주택 경기 불확실성이 확대되고 있다. 특히 매수 수요 위축으
로 주택 매매 거래량이 급감하면서 향후 주택 경기에 대한 부정적 시각이 팽배하다. 무엇보다 여전히 
높은 금리가 부담으로 작용하고 있다. 주택 경기 불황기에 고금리로 인한 대출 부담은 주택 수요를 크
게 위축시킬 수밖에 없기 때문이다. 
 
그림Ⅰ-11. 매수우위지수 추이  표Ⅰ-2. 주요 금융기관 주택담보대출 금리(2024년 2월) 
 
 
상품명 
당월 
최

In [30]:

loader = PyMuPDFLoader("./data/2024_KB_부동산_보고서_최종.pdf")
pages = loader.load_and_split()
print('청크의 수:', len(pages))

청크의 수: 83


In [31]:
pages[10]

Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'source': './data/2024_KB_부동산_보고서_최종.pdf', 'file_path': './data/2024_KB_부동산_보고서_최종.pdf', 'total_pages': 84, 'format': 'PDF 1.5', 'title': 'Morning Meeting', 'author': '손은경', 'subject': '', 'keywords': '', 'moddate': '2024-03-04T15:30:01+09:00', 'trapped': '', 'modDate': "D:20240304153001+09'00'", 'creationDate': "D:20240304153001+09'00'", 'page': 11}, page_content='5 \n2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 \n표Ⅰ-1. 공급 확대 정책 주요 내용 \n날짜 \n주요 내용 \n2023년 9월 \n공공 부문 공급물량 확대: 3기 신도시를 포함한 공급물량 확대 및 조기 공급 방안 마련 \n사업 여건 개선: 전매 제한 완화 및 규제 정상화, 조기 인허가 인센티브 및 절차 개선, 공사비 증액 \n기준 마련 및 인력 확충, 분양 사업의 임대 사업 전환 촉진 \n원활한 자금 지원: PF대출 보증 확대, 중도금 대출 지원 \n비아파트 자금 조달 지원 및 규제 개선 \n도심 공급 기반 확충: 정비사업 절차 및 소규모 사업 사업성 개선 \n2024년 1월 \n도심 공급 확대: 재건축·재개발 패스트트랙 도입 및 재건축 부담금 합리화, 1기 신도시 재정비사업의 \n신속하고 내실 있는 계획 수립, 소규모 정비·도심 복합 사업 속도 개선 및 자금 지원 강화 \n다양한 유형의 주택 공급 확대: 도시·건축 규제 완화 및 세제·금융 지원, 등

In [33]:
%%time
loader = PDFPlumberLoader("./data/2024_KB_부동산_보고서_최종.pdf")
pages = loader.load_and_split()
print('청크의 수:', len(pages))

청크의 수: 83
CPU times: total: 29.5 s
Wall time: 29.9 s


In [34]:
pages[10]

Document(metadata={'source': './data/2024_KB_부동산_보고서_최종.pdf', 'file_path': './data/2024_KB_부동산_보고서_최종.pdf', 'page': 11, 'total_pages': 84, 'Title': 'Morning Meeting', 'Author': '손은경', 'Creator': 'Microsoft® Word 2016', 'CreationDate': "D:20240304153001+09'00'", 'ModDate': "D:20240304153001+09'00'", 'Producer': 'Microsoft® Word 2016'}, page_content='2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망\n표Ⅰ-1. 공급 확대 정책 주요 내용\n날짜 주요 내용\n공공 부문 공급물량 확대: 3기 신도시를 포함한 공급물량 확대 및 조기 공급 방안 마련\n사업 여건 개선: 전매 제한 완화 및 규제 정상화, 조기 인허가 인센티브 및 절차 개선, 공사비 증액\n기준 마련 및 인력 확충, 분양 사업의 임대 사업 전환 촉진\n2023년 9월\n원활한 자금 지원: PF대출 보증 확대, 중도금 대출 지원\n비아파트 자금 조달 지원 및 규제 개선\n도심 공급 기반 확충: 정비사업 절차 및 소규모 사업 사업성 개선\n도심 공급 확대: 재건축·재개발 패스트트랙 도입 및 재건축 부담금 합리화, 1기 신도시 재정비사업의\n신속하고 내실 있는 계획 수립, 소규모 정비·도심 복합 사업 속도 개선 및 자금 지원 강화\n다양한 유형의 주택 공급 확대: 도시·건축 규제 완화 및 세제·금융 지원, 등록 임대 사업 여건 개선\n2024년 1월 및 기업형 장기임대 활성화, 신축 매입 약정 확대, 전세 사기 예방 및 피해 지원\n신도시 등 공공주택 공급: 2024년 공공주택 14만 호 이상 공급, 신도시 조성 속도 제고\n건설 경기 활력 회복: PF대출 지원 및 유동성 지원, 공공 임대 참여 지분 조기 매각, 민간 사업장\

In [36]:
print(pages[10].page_content)

2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망
표Ⅰ-1. 공급 확대 정책 주요 내용
날짜 주요 내용
공공 부문 공급물량 확대: 3기 신도시를 포함한 공급물량 확대 및 조기 공급 방안 마련
사업 여건 개선: 전매 제한 완화 및 규제 정상화, 조기 인허가 인센티브 및 절차 개선, 공사비 증액
기준 마련 및 인력 확충, 분양 사업의 임대 사업 전환 촉진
2023년 9월
원활한 자금 지원: PF대출 보증 확대, 중도금 대출 지원
비아파트 자금 조달 지원 및 규제 개선
도심 공급 기반 확충: 정비사업 절차 및 소규모 사업 사업성 개선
도심 공급 확대: 재건축·재개발 패스트트랙 도입 및 재건축 부담금 합리화, 1기 신도시 재정비사업의
신속하고 내실 있는 계획 수립, 소규모 정비·도심 복합 사업 속도 개선 및 자금 지원 강화
다양한 유형의 주택 공급 확대: 도시·건축 규제 완화 및 세제·금융 지원, 등록 임대 사업 여건 개선
2024년 1월 및 기업형 장기임대 활성화, 신축 매입 약정 확대, 전세 사기 예방 및 피해 지원
신도시 등 공공주택 공급: 2024년 공공주택 14만 호 이상 공급, 신도시 조성 속도 제고
건설 경기 활력 회복: PF대출 지원 및 유동성 지원, 공공 임대 참여 지분 조기 매각, 민간 사업장
공공 인수, 재정 조기 집행 및 민자 사업 확대
자료: 국토교통부 보도자료 요약
■ 2024년 주택시장 3대 변수는 공급과 금리, 그리고 정책
주택시장이 상승과 하락을 반복하면서 주택 경기 불확실성이 확대되고 있다. 특히 매수 수요 위축으
로 주택 매매 거래량이 급감하면서 향후 주택 경기에 대한 부정적 시각이 팽배하다. 무엇보다 여전히
높은 금리가 부담으로 작용하고 있다. 주택 경기 불황기에 고금리로 인한 대출 부담은 주택 수요를 크
게 위축시킬 수밖에 없기 때문이다.
그림Ⅰ-11. 매수우위지수 추이 표Ⅰ-2. 주요 금융기관 주택담보대출 금리(2024년 2월)
당월 당월 전월
상품명
최저 최고 평균
KB주택담보대출 4.24 5.14 4.85
신한주

CSVLoader

In [37]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

In [41]:
# CSV 파일 로더 초기화

# loader = CSVLoader("./data/서울시_부동산_실거래가_정보.csv")

# 위와 같이 수행할 경우 cp949 인코딩 문제 발생하기 때문에 아래와 같이 인코딩 옵션 지정
loader = CSVLoader(
    file_path="./data/서울시_부동산_실거래가_정보.csv",
    encoding="utf-8-sig"
)

documents = loader.load()       # CSV 파일 로드 및 행 분할

print('청크의 수:', len(documents))

청크의 수: 2001


In [46]:
%%time
# UnstructuredCSVLoader 사용
# 'Single' 또는 'elements' 모드 지정 가능 : CSV 파일 전체를 하나의 청크로 만들지, 각 행을 개별 청크로 만들지 선택 가능
# single : 하나의 청크, elements : 각 행을 개별 청크

# CSV 파일 로더 초기화
loader = UnstructuredCSVLoader("./data/서울시_부동산_실거래가_정보.csv", mode='elements')
documents = loader.load()       #

print('청크의 수:', len(documents))

청크의 수: 1
CPU times: total: 688 ms
Wall time: 665 ms


In [47]:
str(documents[0].metadata)[:500]

'{\'source\': \'./data/서울시_부동산_실거래가_정보.csv\', \'file_directory\': \'./data\', \'filename\': \'서울시_부동산_실거래가_정보.csv\', \'last_modified\': \'2025-12-25T16:41:46\', \'text_as_html\': "<table><tr><td>접수연도</td><td>자치구코드</td><td>자치구명</td><td>법정동코드</td><td>법정동명</td><td>지번구분</td><td>지번구분명</td><td>본번</td><td>부번</td><td>건물명</td><td>계약일</td><td>물건금액(만원)</td><td>건물면적(㎡)</td><td>토지면적(㎡)</td><td>층</td><td>권리구분</td><td>취소일</td><td>건축년도</td><td>건물용도</td><td>신고구분</td><td>신고한 개업공인중개사 시군구명</td></tr><tr><td>2024</td><td>11440</td><td>'

In [48]:
str(documents[0].page_content)[:500]

'접수연도 자치구코드 자치구명 법정동코드 법정동명 지번구분 지번구분명 본번 부번 건물명 계약일 물건금액(만원) 건물면적(㎡) 토지면적(㎡) 층 권리구분 취소일 건축년도 건물용도 신고구분 신고한 개업공인중개사 시군구명 2024 11440 마포구 11100 신수동 1 대지 228 3 마인빌 20241031 37300 32.48 20 4 2022 연립다세대 직거래 2024 11320 도봉구 10800 도봉동 1 대지 565 9 (565-9) 20241031 9900 25.92 21 4 2003 연립다세대 직거래 2024 11500 강서구 10900 방화동 1 대지 620 214 (620-214) 20241031 10000 27.37 40.37 7 2011 오피스텔 중개거래 서울 강서구 2024 11680 강남구 11200 자곡동 1 대지 658 0 강남힐스테이트에코 20241031 17000 24.118 31.983 10 2014 오피스텔 중개거래 서울 강남구 2024 11500 강서구 1'

In [50]:
from IPython.display import display, HTML
display(HTML(documents[0].metadata['text_as_html'][:2000]))

Text Splitter

 - RecursiveCharacterTextSplitter : 길이와 구분자로 분할 (사이즈 지정)

 - SemanticChunker : 의미 기반으로 분할 (백분위수, 표준편차, 사분위수 방식)

   · 백분위수 방식 : 코사인 거리 값이 설정한 백분위수를 초과하는 지점을 분할지점으로 선택

   · 표준편차 방식 : 코사인 거리가 평균보다 특정 표준편차 이상 크게 떨어진 지점을 분할지점으로 선택

   · 사분위수 방식 : 코사인 거리의 사분위 범위에 따라 분할지점을 결정

In [67]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PyPDFLoader를 사용하여 PDF 파일 로드
loader = PyPDFLoader("./data/국민주택채권 자주하는 질문(FAQ).pdf")
pages = loader.load()

# PDF 파일의 모든 페이지에서 텍스트를 추출하여 총 글자 수 계산
print('총 글자 수:', len(''.join([i.page_content for i in pages])))


# RecursiveCharacterTextSplitter 초기화
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(pages)
print('분할된 청크의 수:', len(texts))

# 1 번째 청크 출력
texts[1]

총 글자 수: 13187
분할된 청크의 수: 46


Document(metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-11-30T10:48:42+09:00', 'author': 'Owner', 'moddate': '2023-11-30T10:48:42+09:00', 'pdfversion': '1.4', 'source': './data/국민주택채권 자주하는 질문(FAQ).pdf', 'total_pages': 28, 'page': 1, 'page_label': '2'}, page_content='목차No.분류질문및답변Page1매입대상자1종 매입대상자☞면허⋅허가⋅인가를 받거나 등기⋅등록을 신청하는 자 또는 국가지 방 자 치 단 체⋅정 부 투 자 기 관 과  건 설 공 사 의  도 급 계 약 을  체 결 하 는  자  등12매입면제1종 매입 면제자☞주택도시기금법 시행령 별표 및 동법 시행규칙 별표1등23매입요율 및 매입금액매 입 요 율   ☞정액제 또는 정률제 적용매 입 금 액   ☞직접계산 또는 자동계산(주택도시기금 포털)44구입처채권 구입처☞7개사 시중은행(우리,국민,농협,신한,하나,대구,부산은행)55보유방법만기보유만 가능한지?☞즉시매도(은행지점,비대면채널)및 만기보유(은행지점)가능66셀프등기인터넷 사이트이용법공동주택 셀프등기시 기금홈페이지 이용방법☞국민주택채권 매입금액 및 즉시 매도시 고객부담금 조회77.고객부담금고객부담금 증감 사유☞(증가)기준금리 등 상승 →채권 유통금리 상승 →채권매도금액 하락 (하락)기준금리 등 하락 →채권 유통금리 하락 →채권매도금액')

In [68]:
texts[1].page_content

'목차No.분류질문및답변Page1매입대상자1종 매입대상자☞면허⋅허가⋅인가를 받거나 등기⋅등록을 신청하는 자 또는 국가지 방 자 치 단 체⋅정 부 투 자 기 관 과  건 설 공 사 의  도 급 계 약 을  체 결 하 는  자  등12매입면제1종 매입 면제자☞주택도시기금법 시행령 별표 및 동법 시행규칙 별표1등23매입요율 및 매입금액매 입 요 율   ☞정액제 또는 정률제 적용매 입 금 액   ☞직접계산 또는 자동계산(주택도시기금 포털)44구입처채권 구입처☞7개사 시중은행(우리,국민,농협,신한,하나,대구,부산은행)55보유방법만기보유만 가능한지?☞즉시매도(은행지점,비대면채널)및 만기보유(은행지점)가능66셀프등기인터넷 사이트이용법공동주택 셀프등기시 기금홈페이지 이용방법☞국민주택채권 매입금액 및 즉시 매도시 고객부담금 조회77.고객부담금고객부담금 증감 사유☞(증가)기준금리 등 상승 →채권 유통금리 상승 →채권매도금액 하락 (하락)기준금리 등 하락 →채권 유통금리 하락 →채권매도금액'

In [69]:
print('1번 청크의 길이:', len(texts[1].page_content))
print('2번 청크의 길이:', len(texts[2].page_content))

1번 청크의 길이: 492
2번 청크의 길이: 476


In [ ]:
# 라이브러리 불러오기
from langchain_experimental.text_splitter import SemanticChunker
# from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings

# SemanticChunker 초기화
text_splitter = SemanticChunker(
    embeddings=HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3"
    )
)

print("... PDF 파일 로드 및 페이지 분할 ...")

# 텍스트를 의미 단위로 분할
chunks = text_splitter.split_documents(pages)
print('분할된 청크의 수:', len(chunks))

... PDF 파일 로드 및 페이지 분할 ...
분할된 청크의 수: 30


In [71]:
chunks[3]

Document(metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-11-30T10:48:42+09:00', 'author': 'Owner', 'moddate': '2023-11-30T10:48:42+09:00', 'pdfversion': '1.4', 'source': './data/국민주택채권 자주하는 질문(FAQ).pdf', 'total_pages': 28, 'page': 3, 'page_label': '4'}, page_content='- 1 -\n1국민주택채권의매입대상자는?□국민주택채권매입대상자는아래와같습니다.ㅇ 면허⋅허가⋅인가를받거나등기⋅등록을신청하는자또는국가지방자치단체⋅정부투자기관과건설공사의도급계약을체결하는자ㅇ 소유권의보존또는이전시에는소유권의보존또는이전을받은당해등기명의자,상속시에는상속인ㅇ 저당권설정시에는저당권설정자,저당권의이전시에는저당권의이전을받은자<관련법규>s 주택도시기금법:제8조(국민주택채권의매입)s 주택도시기금법시행령:제8조(국민주택채권의매입)및별표')

In [73]:
# <백분위수 방식의 SemanticChunker>
text_splitter = SemanticChunker(
    HuggingFaceEmbeddings(model_name="BAAI/bge-m3"),
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
)
chunks = text_splitter.split_documents(pages)
print('분할된 청크의 수:', len(chunks))

분할된 청크의 수: 30


In [74]:
# <표준편차 방식의 SemanticChunker>
text_splitter = SemanticChunker(
    HuggingFaceEmbeddings(model_name="BAAI/bge-m3"),
    breakpoint_threshold_type="standard_deviation",
    breakpoint_threshold_amount=3,
)
chunks = text_splitter.split_documents(pages)
print('분할된 청크의 수:', len(chunks))

분할된 청크의 수: 28


In [75]:
# <사분위수 방식의 SemanticChunker>
text_splitter = SemanticChunker(
    HuggingFaceEmbeddings(model_name="BAAI/bge-m3"),
    breakpoint_threshold_type="interquartile",
    breakpoint_threshold_amount=1.5,
)
chunks = text_splitter.split_documents(pages)
print('분할된 청크의 수:', len(chunks))

분할된 청크의 수: 29


VectorDB

In [76]:
# 라이브러리 불러오기
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# PyPDFLoader를 사용하여 PDF 파일 로드
loader = PyPDFLoader("./data/국민주택채권 자주하는 질문(FAQ).pdf")
pages = loader.load()

print("청크의 수:", len(pages))

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
print("분할된 청크의 수:", len(splits))

# 각 청크의 길이(문자 수)를 저장한 리스트 생성
chunk_lengths = [len(chunk.page_content) for chunk in splits]
max_length = max(chunk_lengths)
min_length = min(chunk_lengths)
avg_length = sum(chunk_lengths) / len(chunk_lengths)

print('청크의 최대 길이 :', max_length)
print('청크의 최소 길이 :', min_length)
print('청크의 평균 길이 :', avg_length)

청크의 수: 28
분할된 청크의 수: 33
청크의 최대 길이 : 999
청크의 최소 길이 : 5
청크의 평균 길이 : 417.24242424242425


In [77]:
# 임베딩 모델 초기화
embedding_function = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Chroma DB 생성 및 데이터 저장
persist_directory = "./db/directory"
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding_function,
    persist_directory=persist_directory
)

print('문서의 수:', vectordb._collection.count())       # 분할된 청크의 수 만큼 문서가 저장되었는지 확인

문서의 수: 33


In [79]:
# similarity_search 메서드 사용
question = "매입요율 산출 방법"
top_three_docs = vectordb.similarity_search(question, k=2)

for i, doc in enumerate(top_three_docs, 1):
    print(f"문서 {i}:")
    print(f"내용: {doc.page_content[:150]}...")
    print(f"메타데이터: {doc.metadata}")
    print('--' * 20)

문서 1:
내용: - 4 -
3매입요율및매입금액산출은?□국민주택채권은매입대상(인·허가등)에따라정액제또는정률제가적용되고있습니다.ㅇ 정액제(3만원∼500만원)-엽총소지허가/사행행위영업허가/주류판매업면허/주류제조업면허/수렵면허/측량업등록/식품영업허가/게임제공업/골프장업의신규등록/화물자동차운송...
메타데이터: {'producer': 'Hancom PDF 1.3.0.542', 'source': './data/국민주택채권 자주하는 질문(FAQ).pdf', 'creator': 'Hwp 2018 10.0.0.13462', 'page': 6, 'moddate': '2023-11-30T10:48:42+09:00', 'total_pages': 28, 'creationdate': '2023-11-30T10:48:42+09:00', 'pdfversion': '1.4', 'page_label': '7', 'author': 'Owner'}
----------------------------------------
문서 2:
내용: - 23 -
19공동주택(서울소재시가표준액1억)을부부가공동명의로취득시주택채권매입기준이어떻게될까요?□매입자의공유지분율에따라산정한시가표준액에해당하는주택구간매입률적용하시면됩니다.각매입자의공유지분율이50%일경우각각시가표준액5천만원에해당하는매입률(19/1,000)이적용욉니다.【...
메타데이터: {'creator': 'Hwp 2018 10.0.0.13462', 'pdfversion': '1.4', 'page_label': '26', 'creationdate': '2023-11-30T10:48:42+09:00', 'total_pages': 28, 'page': 25, 'moddate': '2023-11-30T10:48:42+09:00', 'source': './data/국민주택채권 자주하는 질문(FAQ).pdf', 'author': 'Owner', 'producer': 'Hancom PDF 1.3.0.542'}
---------------------------------

In [80]:
# similarity_search_with_relevance_scores 메서드 사용
question = "매입요율 산출 방법"
top_three_docs = vectordb.similarity_search_with_relevance_scores(question, k=3)

for i, doc in enumerate(top_three_docs, 1):
    print(f"문서 {i}:")
    print(f"유사 점수 {doc[1]}:")
    print(f"내용: {doc[0].page_content[:150]}...")
    print(f"메타데이터: {doc[0].metadata}")
    print('--' * 20)

문서 1:
유사 점수 0.4854874999156824:
내용: - 4 -
3매입요율및매입금액산출은?□국민주택채권은매입대상(인·허가등)에따라정액제또는정률제가적용되고있습니다.ㅇ 정액제(3만원∼500만원)-엽총소지허가/사행행위영업허가/주류판매업면허/주류제조업면허/수렵면허/측량업등록/식품영업허가/게임제공업/골프장업의신규등록/화물자동차운송...
메타데이터: {'creationdate': '2023-11-30T10:48:42+09:00', 'page_label': '7', 'pdfversion': '1.4', 'moddate': '2023-11-30T10:48:42+09:00', 'source': './data/국민주택채권 자주하는 질문(FAQ).pdf', 'page': 6, 'creator': 'Hwp 2018 10.0.0.13462', 'author': 'Owner', 'total_pages': 28, 'producer': 'Hancom PDF 1.3.0.542'}
----------------------------------------
문서 2:
유사 점수 0.4652790554322245:
내용: - 23 -
19공동주택(서울소재시가표준액1억)을부부가공동명의로취득시주택채권매입기준이어떻게될까요?□매입자의공유지분율에따라산정한시가표준액에해당하는주택구간매입률적용하시면됩니다.각매입자의공유지분율이50%일경우각각시가표준액5천만원에해당하는매입률(19/1,000)이적용욉니다.【...
메타데이터: {'page': 25, 'creator': 'Hwp 2018 10.0.0.13462', 'page_label': '26', 'creationdate': '2023-11-30T10:48:42+09:00', 'source': './data/국민주택채권 자주하는 질문(FAQ).pdf', 'total_pages': 28, 'moddate': '2023-11-30T10:48:42+09:00', 'producer': 'Hancom PDF 1.3.0.542', 'author': 'Owner', 'p

RAG Chat Bot

### 1. Indexing 단계 : 문서 수집, 검색이 용이하도록 DB에 적재하는 과정 

  [Documents > 청크 분할기 > 청크 > DB]
  1) 문서 준비 및 분할 : 문서 수집, 작은 조각(Chunk)로 나눔. 청크는 의미 단위로 나누어지며, 적절한 크기 설정이 중요.
  2) 임베딩 생성 및 저장 : 청크로 나뉜 각 문서를 임베딩 모델을 사용해 벡터 형태로 변환. 
  3) 데이터베이스 적재 및 관리 : 벡터 DB에 저장하여 검색 효율을 높임.

### 2. Query 단계 : 사용자가 질문을 입력하면 관련 정보를 실시간으로 검색하고 답변을 생성하는 단계 

  [질문 > 임베딩 > 임베딩 벡터화 > 유사도 계산 (DB) > (from DB)유사도 top3 > 챗봇 답변]
  1) 질문 입력 및 변환 : 질문을 벡터로 변환.
  2) 검색 및 재정렬 : 변환된 질문 벡터는 DB에서 관련성 높은 문서 청크를 검색하는데 사용. 유사도 점수에 따라 상위 k개 청크 선택.
  3) 프롬프트 템플릿 설정 : 적절한 답변 생성을 위한 프롬프트 템플릿 설정.
  4) 문맥 구성 : 생성모델이 가진 토큰 수 제한을 고려해 필요한 부분만 포함하여 최적의 답변 생성
  5) 답변 생성 및 응답 제공 : 생성된 답변은 필요한 형식과 스타일로 정제하여 최종 응답으로 제공

In [83]:
# 라이브러리 로드
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
# PDF 문서 로드 및 텍스트 분할
loader = PyPDFLoader("./data/국민주택채권 자주하는 질문(FAQ).pdf")
documents = loader.load()  # 문서 로드

# 텍스트 분할 설정: 청크 크기와 겹침 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# 분할된 청크 수
print('분할된 청크의 수:', len(chunks))

분할된 청크의 수: 33


In [88]:
# Embedding 생성 및 Chroma DB에 저장
embedding_function = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
print(f'[1/ ]임베딩 모델 초기화 완료.')

persist_directory = "./db/directory/chroma"
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    persist_directory=persist_directory
)
print('[2/ ]Chroma DB 생성 및 데이터 저장 완료.')
print('     문서의 수:', vectorstore._collection.count())       # 분할된 청크의 수 만큼 문서가 저장되었는지 확인

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  # 검색 및 재정렬 셋팅 : 관련 문서 상위 3개 검색 설정
print('[3/ ]Retriever 설정 완료.')


# 프롬프트 템플릿 설정: 사용자 질문에 대한 답변을 생성하기 위한 템플릿
template = """
당신은 국민주택채권 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 한국어로만 답변해주세요.

컨텍스트: {context}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("placeholder", "{chat_history}"),
        ("human", "{question}")
    ]
)  # 템플릿 초기화

print('[4/ ]프롬프트 템플릿 설정 완료.')

# 모델 설정
model = llm = ChatOpenAI(
    model="qwen2.5-3b-instruct-q4_k_m.gguf",
    base_url="http://localhost:8002/v1",
    api_key="EMPTY",   # 실제로 사용되지 않음
    temperature=0.2,
)

print('[5/ ]언어 모델 설정 완료.')

# 프롬프트 출력 테스트
print(prompt.format(
    context="컨텍스트 예시", 
    chat_history=["대화 기록 예시1", "대화 기록 예시2"], 
    question="질문 예시")
)

print('[6/ ]프롬프트 출력 테스트 완료.')

# 체인 설정
chain = (RunnablePassthrough.assign(context = lambda x: format_docs(retriever.invoke(x["question"])))# 검색된 문서들을 컨텍스트로 포맷팅
    | prompt 
    | model
    | StrOutputParser()     # 결과를 문자열로 반환
)

print('[7/ ]체인 설정 완료.')

# 메모리 설정
chat_history = ChatMessageHistory()
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,  # 세션 ID에 따른 대화 기록 관리
    input_messages_key="question",
    history_messages_key="chat_history",
)

print('[8/ ]메모리 설정 완료.')

# 챗봇 실행 함수 정의
def chat_with_bot():
    session_id = "user_session"
    print("국민채권 전문가 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)")
    while True:
        user_input = input("사용자: ")
        if user_input.lower() == 'quit':
            break

        response = chain_with_memory.invoke(
            {"question": user_input},
            {"configurable": {"session_id": session_id}}
        )

        print("챗봇:", response)

print('[9/ ]챗봇 실행 함수 정의 완료.')

# 메인 실행
if __name__ == "__main__":
    chat_with_bot()

[1/ ]임베딩 모델 초기화 완료.
[2/ ]Chroma DB 생성 및 데이터 저장 완료.
     문서의 수: 33
[3/ ]Retriever 설정 완료.
[4/ ]프롬프트 템플릿 설정 완료.
[5/ ]언어 모델 설정 완료.
System: 
당신은 국민주택채권 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 한국어로만 답변해주세요.

컨텍스트: 컨텍스트 예시

Human: 대화 기록 예시1
Human: 대화 기록 예시2
Human: 질문 예시
[6/ ]프롬프트 출력 테스트 완료.
[7/ ]체인 설정 완료.
[8/ ]메모리 설정 완료.
[9/ ]챗봇 실행 함수 정의 완료.
국민채권 전문가 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)


: 

: 

ChatBOT WEB APP 생성

In [ ]:
# 02_RAG_Chatbot_app.py
import os
import streamlit as st
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 변경
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser  # 변경
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory  # 변경


# PDF 처리 함수
@st.cache_resource
def process_pdf():
    loader = PyPDFLoader("./data/국민주택채권 자주하는 질문(FAQ).pdf")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return text_splitter.split_documents(documents)

# 벡터 스토어 초기화
@st.cache_resource
def initialize_vectorstore():
    chunks = process_pdf()
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
    return Chroma.from_documents(chunks, embeddings)

# 체인 초기화
@st.cache_resource
def initialize_chain():
    vectorstore = initialize_vectorstore()
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    template = """당신은 국민주택채권 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.

    컨텍스트: {context}
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("placeholder", "{chat_history}"),
        ("human", "{question}")
    ])

    model = ChatOpenAI(model_name="qwen2.5-3b-instruct-q4_k_m.gguf", temperature=0.2, base_url="http://localhost:8002/v1", api_key="EMPTY")

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    base_chain = (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["question"]))
        )
        | prompt
        | model
        | StrOutputParser()
    )

    return RunnableWithMessageHistory(
        base_chain,
        lambda session_id: ChatMessageHistory(),
        input_messages_key="question",
        history_messages_key="chat_history",
    )

# Streamlit UI
def main():
    st.set_page_config(page_title="국민주택채권 챗봇", page_icon="🏠")
    st.title("🏠 국민주택채권 AI 어드바이저")
    st.caption("국민주택채권 자주하는 질문(FAQ) 기반 질의응답 시스템")

    # 세션 상태 초기화
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # 채팅 기록 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # 사용자 입력 처리
    if prompt := st.chat_input("국민주택채권 관련 질문을 입력하세요"):
        # 사용자 메시지 표시
        with st.chat_message("user"):
            st.markdown(prompt)
        st.session_state.messages.append({"role": "user", "content": prompt})

        # 체인 초기화
        chain = initialize_chain()

        # AI 응답 생성
        with st.chat_message("assistant"):
            with st.spinner("답변 생성 중..."):
                response = chain.invoke(
                    {"question": prompt},
                    {"configurable": {"session_id": "streamlit_session"}}
                )
                st.markdown(response)

        st.session_state.messages.append({"role": "assistant", "content": response})

if __name__ == "__main__":
    main()

In [3]:
# <스트림릿 적용>

# ngrok 인증키 설정
# !ngrok config add-authtoken 2tnwXk5jQ5uWVBnwy3Ou8Mdmu8v_5eAtrJHxGZRiGGKqsqHsh

# 터널링 및 실행
from pyngrok import ngrok

public_url = ngrok.connect(8501)  # Streamlit 기본 포트
print("앱 접속 URL:", public_url)

앱 접속 URL: NgrokTunnel: "https://fa1353abc118.ngrok-free.app" -> "http://localhost:8501"


In [4]:
# !streamlit run ./02_RAG_Chatbot_app.py

^C
